In [ ]:
!pip install textblob
!python -m textblob.download_corpora
import re
import datetime
from nltk.chat.util import Chat, reflections
from textblob import TextBlob
import tkinter as tk
from tkinter import scrolledtext

# ========== MEMORY AND CHAT HISTORY ==========
user_memory = {"name": None, "mood": None, "color": None, "hobby": None}
chat_history = []

# ========== CHATBOT PAIRS WITH MORE TOPICS ==========
pairs = [
    ['my name is (.*)', ['Nice to meet you, %1!', 'Hello %1, how can I assist you today?']],
    ['i am feeling (.*)', ["Why do you feel %1?", "I hope you feel better soon if you're feeling %1."]],
    ['my favorite color is (.*)', ["%1 is a beautiful color!", "I like %1 too!"]],
    ['i like (.*)', ["%1 sounds fun!", "Tell me more about %1."]],
    ['how are you ?', ['I am a bot, always ready to chat!', 'Doing well, thanks for asking!']],
    ['what is your name ?', ['I am your friendly chatbot.']],
    ['what can you do ?', ['I can chat with you, remember your preferences, and try to understand your mood!']],
    ['tell me a joke', ['Why did the programmer quit his job? Because he didn’t get arrays!', 'Why do programmers prefer dark mode? Because light attracts bugs!']],
    ['what is the weather like?', ['I am a bot and can’t check the weather, but I hope it’s nice where you are!']],
    ['thank you', ['You’re welcome!', 'Glad to help!']],
    ['who are you?', ['I am a friendly chatbot built in Python!']],
    ['help', ['You can tell me your name, mood, favorite color, or hobby. Type "clear" to reset memory or "quit" to exit.']],
    ['clear', ['Memory cleared. Let’s start fresh!']],
    ['quit', ['Goodbye! It was nice talking to you.']],
    ['(hi|hello|hey|hola)', ['Hello!', 'Hi there!', 'Hey!']],
    ['(.*)(location|city)?', ['I am from the cloud, so everywhere and nowhere.']],
    ['(.*) created you?', ['A Python programmer created me.']],
]
pairs += [
    ['what is your favorite book?', ['I love reading science fiction books!', 'I enjoy classic novels. What about you?']],
    ['do you like music?', ['Yes! I like all kinds of music.', 'Music is the soul of life, isn’t it?']],
    ['tell me a fun fact', ['Did you know honey never spoils?', 'Octopuses have three hearts!']],
    ['how old are you?', ['I am timeless.', 'Age is just a number for a bot like me!']],
    ['can you help me with python?', ['Absolutely! I can answer some programming questions or chat about Python.']],
    ['thank you', ['You’re welcome!', 'Anytime! Happy to help.']],
    ['bye', ['Goodbye! Have a great day.', 'See you later!']],
    ['what is AI?', ['AI stands for Artificial Intelligence, which is about making machines smart.', 'Artificial Intelligence is the future!']],
    ['do you have feelings?', ['I don’t have feelings like humans, but I try to understand you.']],
    ['where do you live?', ['I live in the cloud!', 'I am everywhere and nowhere.']],
    ['who is your creator?', ['A skilled Python programmer created me.']],
    ['what languages do you speak?', ['I speak Python fluently!', 'I understand many languages, but Python is my favorite.']],
    ['what is the meaning of life?', ['42! Just kidding. It’s what you make of it.', 'That’s a deep question! What do you think?']],
    ['how do you work?', ['I use patterns and some AI tricks to chat with you.', 'I analyze your text and respond based on what I learned.']],
    ['tell me a joke', ['Why do programmers prefer dark mode? Because light attracts bugs!', 'Why did the computer show up at work late? It had a hard drive!']],
    ['what is your favorite color?', ['I like all colors, but blue feels calming.', 'Colors are beautiful, aren’t they?']],
    ['do you like games?', ['I love games! Especially ones with puzzles.', 'Games are a great way to have fun and learn.']],
    ['what time is it?', ['I don’t have a clock, but your device probably does!', 'Time is relative, but I’m here anytime you need.']],
    ['are you a robot?', ['Yes, I am a chatbot robot.', 'Robots are my family!']],
    ['how can I improve my coding skills?', ['Practice regularly and build projects!', 'Read code from others and keep experimenting.']],
    ['do you like books or movies?', ['Both! Books let you imagine, movies show you new worlds.', 'Depends on my mood, how about you?']],
    ['what is your favorite food?', ['I don’t eat, but I hear pizza is great!', 'If I could eat, I’d try all foods.']],
]


# ========== CHATBOT CLASS WITH MEMORY AND SENTIMENT ==========
class EnhancedChat(Chat):
    def respond(self, user_input):
        user_input_lower = user_input.lower()

        # Commands
        if user_input_lower.strip() == 'clear':
            user_memory.update({k: None for k in user_memory})
            return "Memory cleared. How can I help you now?"

        if user_input_lower.strip() == 'quit':
            return "Goodbye! It was nice talking to you."

        if user_input_lower.strip() == 'help':
            return ("You can tell me your name, mood, favorite color, or hobby. "
                    "Commands: 'clear' to reset memory, 'quit' to exit.")

        # Memory extraction
        m = re.match(r'.*my name is (.*)', user_input_lower)
        if m:
            name = m.group(1).strip().capitalize()
            user_memory['name'] = name
            return f"Nice to meet you, {name}!"

        m = re.match(r'.*i am feeling (.*)', user_input_lower)
        if m:
            mood = m.group(1).strip()
            user_memory['mood'] = mood
            return f"I see you're feeling {mood}. I hope your day gets better!"

        m = re.match(r'.*my favorite color is (.*)', user_input_lower)
        if m:
            color = m.group(1).strip()
            user_memory['color'] = color
            return f"{color.capitalize()} is a beautiful color!"

        m = re.match(r'.*i like (.*)', user_input_lower)
        if m:
            hobby = m.group(1).strip()
            user_memory['hobby'] = hobby
            return f"{hobby.capitalize()} sounds like fun! Tell me more."

        # Sentiment analysis
        sentiment = TextBlob(user_input).sentiment.polarity
        if sentiment < -0.5:
            return "I'm sorry to hear that. Want to talk more about it?"
        elif sentiment > 0.5:
            return "That's great! Glad you're feeling good."

        # Default/fallback
        response = super().respond(user_input)
        if response:
            if user_memory['name']:
                response = response.replace('%1', user_memory['name'])
            else:
                response = response.replace('%1', 'there')
            return response

        return "Sorry, I don't quite understand. Type 'help' for assistance."

# ========== GUI CLASS ==========
class ChatbotGUI:
    def __init__(self):
        self.bot = EnhancedChat(pairs, reflections)
        self.root = tk.Tk()
        self.root.title("My Chatbot")
        self.root.geometry("700x500")
        self.root.configure(bg="#f0f0f0")

        # Optional: add icon if you have one (icon.ico or icon.png in the same folder)
        try:
            self.root.iconbitmap('icon.ico')  # Windows icon format
        except Exception:
            # For PNG icon support you can install Pillow (pip install Pillow)
            try:
                from PIL import Image, ImageTk
                icon = ImageTk.PhotoImage(file='icon.png')
                self.root.call('wm', 'iconphoto', self.root._w, icon)
            except Exception:
                pass

        # Title label
        title_label = tk.Label(self.root, text="My Chatbot", font=("Helvetica", 18, "bold"), bg="#f0f0f0")
        title_label.pack(pady=5)

        # Chat display window
        self.chat_window = scrolledtext.ScrolledText(self.root, state='disabled', width=70, height=20,
                                                     wrap='word', bg='white', fg='black', font=("Arial", 12))
        self.chat_window.pack(padx=10, pady=10)

        # Input frame
        frame = tk.Frame(self.root, bg="#f0f0f0")
        frame.pack(padx=10, pady=5, fill='x')

        self.user_input = tk.Entry(frame, width=55, bg='white', fg='black', font=("Arial", 12))
        self.user_input.pack(side='left', padx=(0, 5), fill='x', expand=True)
        self.user_input.bind("<Return>", self.send_message)

        self.send_button = tk.Button(frame, text="Send", command=self.send_message,
                                     bg="#4CAF50", fg="white", font=("Helvetica", 12, "bold"))
        self.send_button.pack(side='left')

        self.display_message("Bot", "👋 Hello! I'm your chatbot. Type 'help' for assistance.")

        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        self.root.mainloop()

    def display_message(self, sender, message):
        self.chat_window.configure(state='normal')
        self.chat_window.insert(tk.END, f"{sender}: {message}\n")
        self.chat_window.configure(state='disabled')
        self.chat_window.yview(tk.END)

    def send_message(self, event=None):
        user_text = self.user_input.get().strip()
        if not user_text:
            return

        self.display_message("You", user_text)
        chat_history.append(("You", user_text))

        if user_text.lower() == 'quit':
            bot_reply = "Goodbye! It was nice talking to you."
            self.display_message("Bot", bot_reply)
            chat_history.append(("Bot", bot_reply))
            self.save_chat_log()
            self.root.after(1000, self.root.destroy)
            return

        bot_reply = self.bot.respond(user_text)
        self.display_message("Bot", bot_reply)
        chat_history.append(("Bot", bot_reply))
        self.user_input.delete(0, tk.END)

    def save_chat_log(self):
        filename = f"chat_log_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
        with open(filename, "w", encoding='utf-8') as f:
            for speaker, msg in chat_history:
                timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                f.write(f"[{timestamp}] {speaker}: {msg}\n")
        print(f"[Chat history saved to '{filename}']")

    def on_close(self):
        self.save_chat_log()
        self.root.destroy()

# ========== RUN PROGRAM ==========
if __name__ == "__main__":
    ChatbotGUI()